In [1]:
import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path
import matplotlib.pyplot as plt
import pydicom
import cv2
import os

import shutil

In [2]:
train_images_path = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'

merged = pl.read_csv('/kaggle/input/rsna24-lsdc-create-dataset/merged_train.csv')
merged

row_id,study_id,series_id,instance_number,series_description,condition,level,x,y,name,study_id_level,x_min,x_max,y_min,y_max
str,i64,i64,i64,str,str,str,f64,f64,str,str,f64,f64,f64,f64
"""100206310_left_neural_foramina…",100206310,2092806862,13,"""Sagittal T1""","""left_neural_foraminal_narrowin…","""l1_l2""",270.34225,148.221459,"""100206310_2092806862_0013""","""100206310_l1_l2""",236.124282,300.124282,115.234966,179.234966
"""100206310_left_neural_foramina…",100206310,2092806862,12,"""Sagittal T1""","""left_neural_foraminal_narrowin…","""l2_l3""",260.177602,191.705532,"""100206310_2092806862_0012""","""100206310_l2_l3""",226.349493,290.349493,157.311014,221.311014
"""100206310_left_neural_foramina…",100206310,2092806862,13,"""Sagittal T1""","""left_neural_foraminal_narrowin…","""l3_l4""",250.176889,234.398551,"""100206310_2092806862_0013""","""100206310_l3_l4""",219.263597,283.263597,203.162615,267.162615
"""100206310_left_neural_foramina…",100206310,2092806862,12,"""Sagittal T1""","""left_neural_foraminal_narrowin…","""l4_l5""",249.241774,274.786914,"""100206310_2092806862_0012""","""100206310_l4_l5""",219.317425,283.317425,244.733579,308.733579
"""100206310_left_neural_foramina…",100206310,2092806862,12,"""Sagittal T1""","""left_neural_foraminal_narrowin…","""l5_s1""",258.80649,319.853318,"""100206310_2092806862_0012""","""100206310_l5_s1""",223.578398,287.578398,288.122178,352.122178
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""998688940_right_subarticular_s…",998688940,2089880748,20,"""Axial T2""","""right_subarticular_stenosis""","""l5_s1""",228.361062,274.350442,"""998688940_2089880748_0020""","""998688940_l5_s1""",218.999955,282.999955,243.997379,307.997379
"""998688940_spinal_canal_stenosi…",998688940,4135322219,9,"""Sagittal T2/STIR""","""spinal_canal_stenosis""","""l2_l3""",284.880866,211.870036,"""998688940_4135322219_0009""","""998688940_l2_l3""",252.880866,316.880866,179.870036,243.870036
"""998688940_spinal_canal_stenosi…",998688940,4135322219,9,"""Sagittal T2/STIR""","""spinal_canal_stenosis""","""l3_l4""",275.638989,255.306859,"""998688940_4135322219_0009""","""998688940_l3_l4""",243.638989,307.638989,223.306859,287.306859


In [3]:
1_974 / 4 

493.5

In [4]:
group_array = np.array([0] * 493 + [1] * 493 + [2] * 493 + [3] * 495)

In [5]:
study_id_group = merged.select('study_id').unique(maintain_order=True).with_columns(group=pl.Series(group_array))
study_id_group

study_id,group
i64,i64
100206310,0
1002894806,0
1004726367,0
1008446160,0
1009445512,0
…,…
987719637,3
991026205,3
991428866,3


In [6]:
merged = merged.join(study_id_group, on='study_id')
merged

row_id,study_id,series_id,instance_number,series_description,condition,level,x,y,name,study_id_level,x_min,x_max,y_min,y_max,group
str,i64,i64,i64,str,str,str,f64,f64,str,str,f64,f64,f64,f64,i64
"""100206310_left_neural_foramina…",100206310,2092806862,13,"""Sagittal T1""","""left_neural_foraminal_narrowin…","""l1_l2""",270.34225,148.221459,"""100206310_2092806862_0013""","""100206310_l1_l2""",236.124282,300.124282,115.234966,179.234966,0
"""100206310_left_neural_foramina…",100206310,2092806862,12,"""Sagittal T1""","""left_neural_foraminal_narrowin…","""l2_l3""",260.177602,191.705532,"""100206310_2092806862_0012""","""100206310_l2_l3""",226.349493,290.349493,157.311014,221.311014,0
"""100206310_left_neural_foramina…",100206310,2092806862,13,"""Sagittal T1""","""left_neural_foraminal_narrowin…","""l3_l4""",250.176889,234.398551,"""100206310_2092806862_0013""","""100206310_l3_l4""",219.263597,283.263597,203.162615,267.162615,0
"""100206310_left_neural_foramina…",100206310,2092806862,12,"""Sagittal T1""","""left_neural_foraminal_narrowin…","""l4_l5""",249.241774,274.786914,"""100206310_2092806862_0012""","""100206310_l4_l5""",219.317425,283.317425,244.733579,308.733579,0
"""100206310_left_neural_foramina…",100206310,2092806862,12,"""Sagittal T1""","""left_neural_foraminal_narrowin…","""l5_s1""",258.80649,319.853318,"""100206310_2092806862_0012""","""100206310_l5_s1""",223.578398,287.578398,288.122178,352.122178,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""998688940_right_subarticular_s…",998688940,2089880748,20,"""Axial T2""","""right_subarticular_stenosis""","""l5_s1""",228.361062,274.350442,"""998688940_2089880748_0020""","""998688940_l5_s1""",218.999955,282.999955,243.997379,307.997379,3
"""998688940_spinal_canal_stenosi…",998688940,4135322219,9,"""Sagittal T2/STIR""","""spinal_canal_stenosis""","""l2_l3""",284.880866,211.870036,"""998688940_4135322219_0009""","""998688940_l2_l3""",252.880866,316.880866,179.870036,243.870036,3
"""998688940_spinal_canal_stenosi…",998688940,4135322219,9,"""Sagittal T2/STIR""","""spinal_canal_stenosis""","""l3_l4""",275.638989,255.306859,"""998688940_4135322219_0009""","""998688940_l3_l4""",243.638989,307.638989,223.306859,287.306859,3


In [7]:
group_num = 3

In [8]:
for name, data in merged.filter(pl.col('group') == group_num).group_by(['study_id', 'series_id'], maintain_order=True):
    study_id = name[0]
    series_id = name[1]
    
    save_path = f'./train_images/{study_id}/{series_id}'
    
    os.makedirs(save_path, exist_ok=True)
    
    series_path = Path(os.path.join(train_images_path, f'{study_id}/{series_id}'))
    series_file_list = list(series_path.glob('*.dcm'))
    
    for fpath in series_file_list:
        dicom = pydicom.dcmread(fpath).pixel_array
        dicom = (dicom - dicom.min()) / (dicom.max() - dicom.min())
        np.save(
            os.path.join(save_path, f'{fpath.stem}.npy'),
            (dicom * 255).astype(np.uint8)
        )

In [9]:
zip_name = '/kaggle/working/train_images'
directory_name = '/kaggle/working/train_images'

shutil.make_archive(zip_name, 'zip', directory_name)

'/kaggle/working/train_images.zip'

In [10]:
!rm -rf '/kaggle/working/train_images'